In [1]:
%load_ext autoreload
%autoreload 2

from kaggle_speech_recog import *

import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
import pickle
import xgboost as xgb

/usr/local/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/usr/local/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# First layer

## Get feature and label matrices

In [2]:
X_vector_len = 16000
train = SpeechList.get_train('train/audio')
XY_train_valid = train.get_group_unknown_spectrogram_X_and_Y(X_vector_len=X_vector_len, spec_v='3', take_log=False, 
                                                             split_noise=True, n_fabricate_noise=1600, 
                                                             split=0.85)

# XY_train_valid = (X_train_known, X_train_unknown, Y_train_known, Y_train_unknown, X_valid, Y_valid)

/usr/local/anaconda3/lib/python3.6/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


SKIPPED train/audio/_background_noise_/README.md File format b'# Ba'... not understood.


/usr/local/anaconda3/lib/python3.6/site-packages/librosa/filters.py:261: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '


## Get logits from trained neural net

In [3]:
path2log = 'logs/Ensemble_graph_04_9models_run_01.log'

In [4]:
X_train_known, X_train_unknown, Y_train_known, Y_train_unknown, X_valid, Y_valid = XY_train_valid
log = pickle.load(open(path2log, 'rb'))

if hasattr(log.t_cnfg, 'ensemble_logs'):
    L_train_known = gather_logits(X_train_known, log_dir='logs', logs=log.t_cnfg.ensemble_logs)
    L_train_unknown = gather_logits(X_train_unknown, log_dir='logs', logs=log.t_cnfg.ensemble_logs)
    L_valid = gather_logits(X_valid, log_dir='logs', logs=log.t_cnfg.ensemble_logs)
    
GraphClass = globals()[log.graph_name]  # Pick up graph class used to train model
graph = GraphClass(log.g_cnfg)  # Load the same graph configuration

if hasattr(log.t_cnfg, 'ensemble_logs'):
    X2_train_known = graph.get_logits(L_train_known, ckp_dir=log.ckp_dir, batch_size=log.t_cnfg.batch_size, annotate=False)  # Use best model in checkpoint directory
    X2_train_unknown = graph.get_logits(L_train_unknown, ckp_dir=log.ckp_dir, batch_size=log.t_cnfg.batch_size, annotate=False)
    X2_valid = graph.get_logits(L_valid, ckp_dir=log.ckp_dir, batch_size=log.t_cnfg.batch_size, annotate=False)
else:
    X2_train_known = graph.get_logits(X_train_known, ckp_dir=log.ckp_dir, batch_size=log.t_cnfg.batch_size, annotate=False)
    X2_train_unknown = graph.get_logits(X_train_unknown, ckp_dir=log.ckp_dir, batch_size=log.t_cnfg.batch_size, annotate=False)
    X2_valid = graph.get_logits(X_valid, ckp_dir=log.ckp_dir, batch_size=log.t_cnfg.batch_size, annotate=False)

INFO:tensorflow:Restoring parameters from checkpoints/SpectrogramMultiLSTMRandomInputModify_graph_03_lr_decay_run_01_group_unknown/best/model-57354
INFO:tensorflow:Restoring parameters from checkpoints/NoveltyDetectionSpectrogramMultiLSTMRandomInputModify_graph_01_run_01/best/model-82750
INFO:tensorflow:Restoring parameters from checkpoints/SpectrogramMultiLSTMRandomInputModify_graph_02_run_01/best/model-90650
INFO:tensorflow:Restoring parameters from checkpoints/SpectrogramMultiLSTMRandomInputModify2_graph_02_lower_lr_lr_decay_run_01_group_unknown/best/model-53088
INFO:tensorflow:Restoring parameters from checkpoints/SpectrogramCLSTMBidirectional_graph_03_run_01/best/model-17450
INFO:tensorflow:Restoring parameters from checkpoints/SpectrogramDSConvDropout_graph_01_run_01/best/model-22740
INFO:tensorflow:Restoring parameters from checkpoints/SpectrogramConcatCLSTM2_graph_01_run_01_group_unknown/best/model-23858
INFO:tensorflow:Restoring parameters from checkpoints/SpectrogramConcatCLS

# Second layer

## Train IsolationForest and OneClassSVM

In [5]:
X2 = np.concatenate((X2_train_known, X2_train_unknown, X2_valid), axis=0)
Y = np.concatenate((Y_train_known, Y_train_unknown, Y_valid), axis=0)
Y_argmax = np.argmax(Y, axis=1)

In [6]:
model_dict_iso_forest = {}
model_dict_one_class_svm = {}

iso_forest_contamination = 0.005
one_class_svm_nu = 0.01

for i in list(set(Y_argmax)):  # [0: # of labels]
    i_X2 = X2[np.where(Y_argmax == i)[0], :]
    
    model_dict_iso_forest[i] = IsolationForest(max_samples=len(i_X2), contamination=iso_forest_contamination).fit(i_X2)
    model_dict_one_class_svm[i] = OneClassSVM(nu=one_class_svm_nu).fit(i_X2)

# Save for later
pickle.dump(model_dict_iso_forest, open('_'.join(['IsolationForests', log.joined_name]), 'wb'))
pickle.dump(model_dict_one_class_svm, open('_'.join(['OneClassSVMs', log.joined_name]), 'wb'))

## Train XGBoost

In [7]:
random_pick_unknown = True

if random_pick_unknown:
    i_train_unknown = np.random.choice(len(X2_train_unknown), size=int(len(X2_train_known) / 11), replace=False)
    random_X2_train_unknown = X2_train_unknown[i_train_unknown, :]
    random_Y_train_unknown = Y_train_unknown[i_train_unknown, :]

    X2_train = np.concatenate((X2_train_known, random_X2_train_unknown))
    Y_train = np.concatenate((Y_train_known, random_Y_train_unknown))
else:
    X2_train = np.concatenate((X2_train_known, X2_train_unknown))
    Y_train = np.concatenate((Y_train_known, Y_train_unknown))
    
Y_train_argmax = np.argmax(Y_train, axis=1)
Y_valid_argmax = np.argmax(Y_valid, axis=1)

In [8]:
xgb_train = xgb.DMatrix(X2_train, label=Y_train_argmax)
xgb_valid = xgb.DMatrix(X2_valid, label=Y_valid_argmax)

params = {'objective': 'multi:softprob',  # labels are 1-dim, not one-hot encoded
          'num_class': Y_train.shape[-1],
          'eval_metric': 'mlogloss', 
          'eta': 0.05,
          'subsample': 0.5,
          'max_depth': 1}
watchlist = [(xgb_train, 'train'), (xgb_valid, 'valid')]

xgboost = xgb.train(params, xgb_train, num_boost_round=1000, evals=watchlist, 
                    early_stopping_rounds=50, verbose_eval=10)

# Save for later
pickle.dump(xgboost, open('_'.join(['XGBoost', log.joined_name]), 'wb'))

[0]	train-mlogloss:2.19578	valid-mlogloss:2.19744
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 50 rounds.
[10]	train-mlogloss:1.03945	valid-mlogloss:1.04728
[20]	train-mlogloss:0.601851	valid-mlogloss:0.612506
[30]	train-mlogloss:0.367786	valid-mlogloss:0.380278
[40]	train-mlogloss:0.232056	valid-mlogloss:0.245855
[50]	train-mlogloss:0.151076	valid-mlogloss:0.16586
[60]	train-mlogloss:0.101926	valid-mlogloss:0.117492
[70]	train-mlogloss:0.071655	valid-mlogloss:0.087732
[80]	train-mlogloss:0.052896	valid-mlogloss:0.06953
[90]	train-mlogloss:0.040938	valid-mlogloss:0.057928
[100]	train-mlogloss:0.033202	valid-mlogloss:0.050776
[110]	train-mlogloss:0.027981	valid-mlogloss:0.045923
[120]	train-mlogloss:0.024376	valid-mlogloss:0.042776
[130]	train-mlogloss:0.021823	valid-mlogloss:0.04053
[140]	train-mlogloss:0.019993	valid-mlogloss:0.038993
[150]	train-mlogloss:0.018593	valid-mlogloss:0.038146
[

## Make extended X2's

In [9]:
list_X2 = [X2_train_known, X2_train_unknown, X2_valid]

# Get X's from sklearn models
list_X2_iso_forest = get_values_with_sklearn_models(list_X2, model_dict_iso_forest)
list_X2_one_class_svm = get_values_with_sklearn_models(list_X2, model_dict_one_class_svm)

# Get X's from xgboost
list_X2_xgboost = get_values_with_xgboost(list_X2, xgboost)

# Concat
#list_L = [L_train_known, L_train_unknown, L_valid]
to_concatenate = [list_X2, list_X2_iso_forest, list_X2_one_class_svm, list_X2_xgboost]
X2ext_train_known, X2ext_train_unknown, X2ext_valid = get_concatenated(to_concatenate)

In [10]:
X2Y_train_valid = (X2ext_train_known, X2ext_train_unknown, Y_train_known, Y_train_unknown, X2ext_valid, Y_valid)

## Train

In [21]:
graph_name = 'Ensemble'  # Specify graph class name

g_cnfg = Config('graph_02_2nd_layer_for_' + log.joined_name)  # Specify graph configuration name
g_cnfg.X_n_features = X2Y_train_valid[0].shape[1]
g_cnfg.Y_vector_len = X2Y_train_valid[-1].shape[1]

g_cnfg.flat_hiddens = [36, 36]

g_cnfg.lr_initial = 0.0001
g_cnfg.lr_decay_steps = 100000000
g_cnfg.lr_decay_rate = 1.0  # 1.0 means no decay

t_cnfg = Config('run_01')  # Specify train configuration name
t_cnfg.ckp_dir = 'checkpoints'
t_cnfg.tb_dir = 'tensorboard'
t_cnfg.log_dir = 'logs'
t_cnfg.le = train.le
t_cnfg.random_pick_unknown = True
t_cnfg.max_step = 100000000
t_cnfg.batch_size = 150
t_cnfg.dropout_keep_prob = 0.5
t_cnfg.log_every = int(int(XY_train_valid[0].shape[0] / 11 * 12) / t_cnfg.batch_size / 2)
t_cnfg.n_ave_ll_valid = 8
t_cnfg.start_step_early_stopping = 100000000
t_cnfg.early_stopping_patience =0.1

In [22]:
print(g_cnfg)
print(t_cnfg)

GraphClass = globals()[graph_name]
graph = GraphClass(g_cnfg)
graph.train_model(t_cnfg, X2Y_train_valid, annotate=False)

name: graph_02_2nd_layer_for_Ensemble_graph_04_9models_run_01
X_n_features: 48
Y_vector_len: 12
flat_hiddens: [36, 36]
lr_decay_rate: 1.0
lr_decay_steps: 100,000,000
lr_initial: 0.0001

name: run_01
batch_size: 150
ckp_dir: checkpoints
dropout_keep_prob: 0.5
early_stopping_patience: 0.1
le: LabelEncoder()
log_dir: logs
log_every: 79
max_step: 100,000,000
n_ave_ll_valid: 8
random_pick_unknown: 1
start_step_early_stopping: 100,000,000
tb_dir: tensorboard

Ensemble_graph_02_2nd_layer_for_Ensemble_graph_04_9models_run_01_run_01
Epoch size is 23,815 | Batch size is 150 | 158 steps per epoch
115 leftover gets discarded at the end of every epoch

Training starts @ 01/16/2018 01:11:41


KeyboardInterrupt: 